In [ ]:
# Install dependencies
!apt-get install ffmpeg -y
!pip install gTTS gradio imageio seaborn matplotlib

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil
import imageio
from diffusers import StableDiffusionPipeline
from gtts import gTTS
import gradio as gr

def load_diffusion_model():
    model = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4-original", torch_dtype=torch.float16)
    model.to("cuda")  # Use GPU if available
    return model

def simulate_animation(file, x_column, y_column, chart_type):
    df = pd.read_csv(file.name)
    os.makedirs("frames", exist_ok=True)

    # Generate frames based on the selected chart type
    for i in range(1, len(df) + 1):
        plt.figure(figsize=(6, 4))

        if chart_type == "Scatter Plot":
            sns.scatterplot(data=df, x=x_column, y=y_column, color='blue')
        elif chart_type == "Line Chart":
            sns.lineplot(data=df, x=x_column, y=y_column, color='blue')
        elif chart_type == "Bar Chart":
            sns.barplot(data=df, x=x_column, y=y_column, color='blue')
        elif chart_type == "Histogram":
            sns.histplot(df[y_column][:i], bins=20, kde=True, color='blue')

        plt.xlim(df[x_column].min(), df[x_column].max())
        plt.ylim(df[y_column].min(), df[y_column].max())
        plt.title(f"{chart_type} of {x_column} vs {y_column}")
        plt.xlabel(x_column)
        plt.ylabel(y_column)
        plt.grid()
        frame_path = f"frames/frame_{i:03}.png"
        plt.savefig(frame_path)
        plt.close()

    # Create video from frames
    images = [imageio.imread(f"frames/frame_{i:03}.png") for i in range(1, len(df) + 1)]
    video_path = "animation.mp4"
    imageio.mimsave(video_path, images, fps=5)

    # Save the final plot image (last frame)
    final_plot_path = "final_plot.png"
    shutil.copy(f"frames/frame_{len(df):03}.png", final_plot_path)

    # Generate audio narration
    summary = f"This is a simulated animation of {y_column} as a function of {x_column}. " \
              f"The minimum {y_column} is {df[y_column].min()}, maximum is {df[y_column].max()}, " \
              f"and average is {df[y_column].mean():.2f}."
    tts = gTTS(text=summary, lang='en')
    audio_path = "graph_narration.mp3"
    tts.save(audio_path)

    shutil.rmtree("frames")
    return video_path, audio_path, final_plot_path

# Gradio interface
gr.Interface(
    fn=simulate_animation,
    inputs=[
        gr.File(label="Upload CSV"),
        gr.Textbox(label="X-Axis Column"),
        gr.Textbox(label="Y-Axis Column"),
        gr.Radio(["Scatter Plot", "Line Chart", "Bar Chart", "Histogram"], label="Chart Type")
    ],
    outputs=[
        gr.Video(label="Animation Only (.mp4)"),
        gr.Audio(label="Narration Only (.mp3)"),
        gr.Image(type="filepath", label="Final Plot Image (.png)")
    ],
    title="AI Based Scientific Research Visualizer",
    description="Upload CSV → get animated video, narration, and the final plot image based on your selected chart type."
).launch()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4b5ae190ac5a93a539.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggi